### Ex 2.7.1
The goal of this exercise is to create a simulated database featuring selection bias, and show how misleading the results can be if controls aren't properly taken to ensure similarity of the tested groups.

### Uncorrelated Case

In [28]:
import random as rand
import pandas as pd
import numpy as np

### First of all let's generate some samples for a case where smoking is unrelated to any other properties
N = 1000 # Let's make 1000 people
P_u = 0.5 # Of which 50% are generally unhealthy
P_s_h = P_s_u = 0.5 # And the probability of being a smoker is independant of health, or we've done RCT to control for this underlying

healths = [rand.random() < P_u for i in range(N)] # Generate random healthy and unhealthy people according to fraction unhealthy
p_smoker = [P_s_h * (healths[i] == 1) + P_s_u * (healths[i] == 0) for i in range(N)] # Assign smoking probabilites according to health (note that odds are same)
smoker = [rand.random() < p_smoker[i] for i in range(N)] # Assign smoking according to probs
outcome = [healths[i]*3 + smoker[i]*(-1) for i in range(N)] # We assign a linear model where being generally healthy is worth 3 points and smoking removes a point.

samples = pd.DataFrame({'Health':healths, 'Smoker':smoker, 'Outcome':outcome})
samples.head()

,Health,Smoker,Outcome
0,True,True,2
1,False,False,0
2,True,True,2
3,True,False,3
4,False,False,0


Given this case we can simply evaluate the expected impact of smoking as being the mean for the smokers vs the non smokers, the naive approach will work just fine.

In [29]:
sample_Smoking = samples[samples['Smoker'] == True]['Outcome']     # Select and take mean of smoking/nonsmoking samples
sample_NonSmoking = samples[samples['Smoker'] == False]['Outcome'] # ...

print(f"Smoking Mean Score: {sample_Smoking.mean()}")                                                          # Sample mean
print(f"Std Deviation of Smoking Mean Score: {np.sqrt(np.var(sample_Smoking)/len(sample_Smoking))}")           # MC variance estimator
print(f"Non-Smoking Mean Score: {sample_NonSmoking.mean()}")                                                   # ...
print(f"Std Deviation of Non Smoking Mean Score: {np.sqrt(np.var(sample_NonSmoking)/len(sample_NonSmoking))}") # ...
print(f"---------------------------------------")
print(f"Estimated Impact of Smoking: {sample_Smoking.mean() - sample_NonSmoking.mean()}")                                                           # Difference of means
print(f"Std Deviation of Difference: {np.sqrt(np.var(sample_NonSmoking)/len(sample_NonSmoking) + np.var(sample_Smoking)/len(sample_Smoking))}")     # Combined variance to std
print(f"Estimate 95% Conf Interval: {np.sqrt(np.var(sample_NonSmoking)/len(sample_NonSmoking) + np.var(sample_Smoking)/len(sample_Smoking))*1.96}") # Combined variance to conf int

Smoking Mean Score: 0.4274661508704062
Std Deviation of Smoking Mean Score: 0.06589275092327597
Non-Smoking Mean Score: 1.4720496894409938
Std Deviation of Non Smoking Mean Score: 0.06824051333881244
---------------------------------------
Estimated Impact of Smoking: -1.0445835385705875
Std Deviation of Difference: 0.09486106833143682
Estimate 95% Conf Interval: 0.18592769392961617


### Correlated Case

Now we can check the other case with very similar code.

In [30]:
### First of all let's generate some samples for a case where smoking is unrelated to any other properties
N = 1000 # Let's make 1000 people
P_u = 0.5 # Of which 50% are generally unhealthy
P_s_h = 0.4 # A bit less likely for healthy people to smoke compared to before
P_s_u = 0.6 # A bit more likely to be a smoker if you're unhealthy

healths = [rand.random() < P_u for i in range(N)] # Generate random healthy and unhealthy people according to fraction unhealthy
p_smoker = [P_s_h * (healths[i] == 1) + P_s_u * (healths[i] == 0) for i in range(N)] # Assign smoking probabilites according to health (note that odds are no longer the same)
smoker = [rand.random() < p_smoker[i] for i in range(N)] # Assign smoking according to probs
outcome = [healths[i]*3 + smoker[i]*(-1) for i in range(N)] # We assign a linear model where being generally healthy is worth 3 points and smoking removes a point.

samples = pd.DataFrame({'Health':healths, 'Smoker':smoker, 'Outcome':outcome})

sample_Smoking = samples[samples['Smoker'] == True]['Outcome']     # Select and take mean of smoking/nonsmoking samples
sample_NonSmoking = samples[samples['Smoker'] == False]['Outcome'] # ...

print(f"Smoking Mean Score: {sample_Smoking.mean()}")                                                          # Sample mean
print(f"Std Deviation of Smoking Mean Score: {np.sqrt(np.var(sample_Smoking)/len(sample_Smoking))}")           # MC variance estimator
print(f"Non-Smoking Mean Score: {sample_NonSmoking.mean()}")                                                   # ...
print(f"Std Deviation of Non Smoking Mean Score: {np.sqrt(np.var(sample_NonSmoking)/len(sample_NonSmoking))}") # ...
print(f"---------------------------------------")
print(f"Estimated Impact of Smoking: {sample_Smoking.mean() - sample_NonSmoking.mean()}")                                                           # Difference of means
print(f"Std Deviation of Difference: {np.sqrt(np.var(sample_NonSmoking)/len(sample_NonSmoking) + np.var(sample_Smoking)/len(sample_Smoking))}")     # Combined variance to std
print(f"Estimate 95% Conf Interval: {np.sqrt(np.var(sample_NonSmoking)/len(sample_NonSmoking) + np.var(sample_Smoking)/len(sample_Smoking))*1.96}") # Combined variance to conf int

Smoking Mean Score: 0.24180327868852458
Std Deviation of Smoking Mean Score: 0.06688830996862616
Non-Smoking Mean Score: 1.79296875
Std Deviation of Non Smoking Mean Score: 0.06501456134258858
---------------------------------------
Estimated Impact of Smoking: -1.5511654713114753
Std Deviation of Difference: 0.09327882501955215
Estimate 95% Conf Interval: 0.1828264970383222


### What if we perform a fit that's allowed to see the health variable, and hopefully assign some blame to it instead


In [ ]:
import statsmodels.api as sm
X = samples[['Smoker','Health']].astype(int) # Create a subframe of our explanatory variables 
X = sm.add_constant(X)                       # Add intercept fitting constant
model = sm.OLS(samples['Outcome'], X).fit()  
model.summary()                              # And now we get a perfect recovery since the model is so simple and noise free!

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Outcome   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 5.008e+32
Date:                Tue, 29 Jul 2025   Prob (F-statistic):               0.00
Time:                        13:16:10   Log-Likelihood:                 32612.
No. Observations:                1000   AIC:                        -6.522e+04
Df Residuals:                     997   BIC:                        -6.520e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       9.593e-16   9.75e-17      9.838      0.000    7.68e-16    1.15e-15
Smoker        -1.0000   1.07e-16  -9.33e+15      0.000      -1.000      -1.000
Health         3.0000   1.07e-16    2.8e+16      0.000       3.000       3.000
==============================================================================
Omnibus:                     3876.674   Durbin-Watson:                   1.805
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              158.811
Skew:                          -0.014   Prob(JB):                     3.27e-35
Kurtosis:                       1.048   Cond. No.                         3.48
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Whats it mean
As expected it's very important in an RCT to properly distribute your samples, or to underrstand and correct for any underlying confounding variables, with potential correlations, cross correlations or interactions. Making sure that your sample groups are as similar as possible in any and every possible interaction, ideally down to a one to one, but at least down to as above correct in distribution is critical to get trustworthy results. If you can't filter out a potential issue, allowing flexibility in it can help reduce issues caused by it's inclusion.